# Market Segmentation with Worldwide COVID-19 Vaccinations

In [ ]:
'''Things to do:
- map plot
-pca
- k means clustering
    - elbow plot
- hierarchical agglomerative clustering
    - dendrogram
- metrics
    - silhouette coefficient
    b- calinski harabasz score

''' 


# Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import metrics

pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('../data/owid-covid-data_updated.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
#remove anything that isn't a country
# create list of things that are not countries 
continents = ['North America', 'Europe', 'Asia','European Union', 
              'Africa', 'South America', 'World', 'United Kingdom']

data = data[~data['location'].isin(continents)]

In [ ]:
data.location.unique()

In [4]:
# drop columns that have been 'smoothed' or have some kind of calculation using 'per'
cols = [data.drop(columns= i, inplace= True) for i in data.columns if 'smoothed' in i]
cols = [data.drop(columns= i, inplace= True) for i in data.columns if 'per' in i]

In [8]:
data.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'total_deaths', 'new_deaths', 'reproduction_rate', 'icu_patients',
       'hosp_patients', 'weekly_icu_admissions', 'weekly_hosp_admissions',
       'new_tests', 'total_tests', 'positive_rate', 'tests_units',
       'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated',
       'new_vaccinations', 'stringency_index', 'population',
       'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
       'extreme_poverty', 'cardiovasc_death_rate', 'diabetes_prevalence',
       'female_smokers', 'male_smokers', 'handwashing_facilities',
       'life_expectancy', 'human_development_index'],
      dtype='object')